In [3]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [42]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [299]:
for k,v in zip(questions,expected):
    print(k,v,len(k),len(v))

    6+5 11   7 4
   4+17 75   7 4
   98+8 97   7 4
  7+446 651  7 4
 632+87 314  7 4
977+445 1323 7 4
   8+97 87   7 4
   3+87 81   7 4
  2+292 294  7 4
   2+35 55   7 4
   5+13 36   7 4
  449+5 949  7 4
    3+7 10   7 4
   8+83 46   7 4
  7+422 231  7 4
851+107 859  7 4
    9+0 9    7 4
 73+154 488  7 4
 28+524 507  7 4
  454+9 463  7 4
  5+161 166  7 4
   5+17 76   7 4
  137+5 736  7 4
  3+757 760  7 4
  7+223 329  7 4
  21+43 46   7 4
  1+578 876  7 4
    4+4 8    7 4
  78+73 124  7 4
    2+0 2    7 4
    4+5 9    7 4
  7+264 469  7 4
   9+82 37   7 4
 84+981 237  7 4
  5+215 517  7 4
  38+65 139  7 4
    1+7 8    7 4
   19+1 92   7 4
  2+643 348  7 4
  894+9 507  7 4
    1+8 9    7 4
    4+8 12   7 4
  125+3 524  7 4
    0+4 4    7 4
  385+2 585  7 4
 823+08 408  7 4
 19+111 202  7 4
   7+02 27   7 4
  35+73 90   7 4
   7+45 61   7 4
398+465 1457 7 4
 35+808 861  7 4
  813+1 319  7 4
  341+1 144  7 4
 82+955 587  7 4
 821+13 159  7 4
  453+6 360  7 4
  2+027 722  7 4
   33+3 36   7

   7+86 75   7 4
  92+29 121  7 4
  74+97 126  7 4
   2+87 80   7 4
  6+257 758  7 4
 74+881 235  7 4
 87+901 187  7 4
 33+157 784  7 4
   08+7 87   7 4
 237+83 770  7 4
 976+68 765  7 4
621+732 363  7 4
   58+9 94   7 4
   0+66 66   7 4
   97+2 81   7 4
  123+5 326  7 4
   23+5 37   7 4
   62+3 29   7 4
 83+757 795  7 4
    5+0 5    7 4
  9+079 979  7 4
  2+959 961  7 4
573+693 771  7 4
  3+461 167  7 4
 764+29 559  7 4
   45+6 60   7 4
  72+51 42   7 4
   4+27 76   7 4
 89+139 1029 7 4
 32+782 310  7 4
   72+0 27   7 4
  6+044 446  7 4
  19+54 136  7 4
  649+9 955  7 4
  25+38 135  7 4
  7+059 957  7 4
   1+71 18   7 4
   99+9 108  7 4
  87+74 125  7 4
 314+01 423  7 4
   3+99 102  7 4
   92+0 29   7 4
   4+42 28   7 4
   0+73 37   7 4
198+318 1704 7 4
291+446 836  7 4
  01+42 34   7 4
  632+5 241  7 4
   91+4 23   7 4
  638+5 841  7 4
   0+25 52   7 4
  819+4 922  7 4
  5+948 854  7 4
  286+2 684  7 4
  25+26 114  7 4
   77+0 77   7 4
  816+5 623  7 4
916+129 1540 7 4
    5+2 7    7

 93+137 770  7 4
678+181 1057 7 4
268+147 1603 7 4
  1+735 538  7 4
 42+886 712  7 4
   77+5 82   7 4
   6+67 82   7 4
 31+394 506  7 4
 656+98 745  7 4
  44+47 118  7 4
  2+047 742  7 4
  069+7 967  7 4
  26+63 98   7 4
   8+04 48   7 4
 517+91 734  7 4
   79+5 102  7 4
  872+9 287  7 4
   0+09 90   7 4
  0+477 774  7 4
 54+616 661  7 4
003+772 577  7 4
  0+927 729  7 4
373+373 746  7 4
075+766 1237 7 4
764+178 1338 7 4
 03+045 570  7 4
  21+06 72   7 4
  44+13 75   7 4
  2+133 333  7 4
  597+8 803  7 4
 57+281 257  7 4
  42+55 79   7 4
   9+29 101  7 4
397+493 1187 7 4
 406+21 616  7 4
  263+3 365  7 4
 326+33 656  7 4
  9+856 667  7 4
   0+54 45   7 4
951+017 869  7 4
593+719 1312 7 4
   2+29 94   7 4
 826+34 671  7 4
  5+845 553  7 4
  5+761 172  7 4
671+638 1012 7 4
444+358 1297 7 4
 896+16 759  7 4
  69+42 120  7 4
 616+75 673  7 4
  37+81 91   7 4
   8+74 55   7 4
016+331 743  7 4
  48+02 104  7 4
  411+6 120  7 4
  2+088 882  7 4
  074+4 474  7 4
   53+8 43   7 4
  12+95 80   7

  06+23 92   7 4
 91+639 955  7 4
  874+5 483  7 4
  687+3 789  7 4
 127+25 773  7 4
  036+3 633  7 4
  192+5 296  7 4
896+202 900  7 4
   74+9 56   7 4
  5+366 668  7 4
  1+552 256  7 4
   0+74 47   7 4
   0+33 33   7 4
   22+7 29   7 4
  4+915 523  7 4
 13+936 670  7 4
 77+854 535  7 4
234+606 1038 7 4
  052+4 254  7 4
 937+56 804  7 4
 955+94 608  7 4
   5+97 84   7 4
 678+44 920  7 4
 72+485 611  7 4
 69+727 823  7 4
  254+7 459  7 4
  5+625 531  7 4
  1+451 155  7 4
 597+65 851  7 4
   9+11 20   7 4
 945+13 580  7 4
 28+126 703  7 4
216+657 1368 7 4
354+836 1091 7 4
783+584 872  7 4
962+129 1190 7 4
   7+18 88   7 4
  42+59 119  7 4
 584+78 572  7 4
742+764 714  7 4
  47+92 103  7 4
 11+833 349  7 4
   88+6 94   7 4
  8+119 919  7 4
 79+556 752  7 4
 309+09 993  7 4
 67+766 743  7 4
   6+62 32   7 4
   2+27 74   7 4
  146+0 641  7 4
   66+8 74   7 4
 78+298 979  7 4
 05+583 435  7 4
  42+27 96   7 4
 836+27 710  7 4
  9+672 285  7 4
   59+6 101  7 4
  5+143 346  7 4
   75+0 57   7

  294+5 497  7 4
  588+7 892  7 4
885+521 713  7 4
  7+616 623  7 4
   14+8 49   7 4
  9+765 576  7 4
   8+95 67   7 4
   3+79 100  7 4
  465+5 569  7 4
  128+0 821  7 4
  4+539 939  7 4
 107+48 785  7 4
 48+569 1049 7 4
955+668 1425 7 4
 03+627 756  7 4
  8+478 882  7 4
  994+2 501  7 4
  11+73 48   7 4
595+955 1154 7 4
 89+746 745  7 4
   12+6 27   7 4
  18+42 105  7 4
  612+3 219  7 4
 75+618 873  7 4
   2+52 27   7 4
 104+42 425  7 4
 86+172 339  7 4
  19+07 161  7 4
 06+861 228  7 4
 389+89 1081 7 4
 18+267 843  7 4
 099+58 1075 7 4
  9+815 527  7 4
  443+0 344  7 4
  1+005 501  7 4
  018+2 812  7 4
  88+55 143  7 4
 952+45 313  7 4
 897+52 823  7 4
  8+999 1007 7 4
 132+42 255  7 4
055+377 1323 7 4
 53+172 306  7 4
  68+98 175  7 4
 81+855 576  7 4
  45+55 109  7 4
   2+83 40   7 4
 369+66 1029 7 4
  69+61 112  7 4
  47+62 100  7 4
461+699 1160 7 4
  79+13 128  7 4
 598+31 908  7 4
  171+1 172  7 4
  278+6 878  7 4
586+194 1176 7 4
  77+47 151  7 4
 23+223 354  7 4
 414+71 431  7

...

 408+94 853  7 4
 76+902 276  7 4
 91+408 823  7 4
 98+187 870  7 4
477+843 1122 7 4
 343+17 414  7 4
  06+51 75   7 4
718+362 1080 7 4
 38+827 811  7 4
 72+718 844  7 4
 556+96 724  7 4
 589+08 1065 7 4
101+225 623  7 4
  694+4 500  7 4
 591+28 277  7 4
614+367 1179 7 4
403+755 861  7 4
 476+15 725  7 4
 194+97 570  7 4
 27+279 1044 7 4
 872+54 323  7 4
256+256 1304 7 4
617+694 1212 7 4
 36+784 550  7 4
284+881 670  7 4
764+138 1298 7 4
  988+0 889  7 4
746+244 1089 7 4
 44+345 587  7 4
  81+29 110  7 4
472+329 1197 7 4
162+408 1065 7 4
831+132 369  7 4
 521+09 215  7 4
 06+841 208  7 4
 388+53 918  7 4
  4+772 281  7 4
285+277 1354 7 4
  645+2 548  7 4
 65+116 667  7 4
 617+06 776  7 4
 13+184 512  7 4
276+085 1252 7 4
222+689 1208 7 4
 681+46 250  7 4
 186+59 776  7 4
 497+99 893  7 4
 035+23 562  7 4
 218+29 904  7 4
 21+541 157  7 4
522+933 564  7 4
 682+61 302  7 4
 374+19 564  7 4
 35+159 1004 7 4
521+325 648  7 4
 407+42 728  7 4
 47+299 1066 7 4
387+982 1072 7 4
138+966 1500 7

 261+46 226  7 4
518+464 1279 7 4
 43+819 952  7 4
 32+096 713  7 4
 36+697 859  7 4
 84+843 396  7 4
 77+028 897  7 4
 205+06 562  7 4
 932+37 312  7 4
486+989 1673 7 4
 788+06 947  7 4
 22+818 840  7 4
 868+36 931  7 4
618+056 1466 7 4
 308+46 867  7 4
 717+54 762  7 4
 76+692 363  7 4
 968+73 906  7 4
  329+4 927  7 4
 786+89 785  7 4
 362+28 345  7 4
 33+964 502  7 4
624+626 1052 7 4
212+596 907  7 4
 632+61 252  7 4
 036+53 665  7 4
662+898 1164 7 4
  281+5 187  7 4
628+191 1017 7 4
 89+978 977  7 4
  841+5 153  7 4
 305+45 557  7 4
  4+991 203  7 4
 053+09 440  7 4
 53+506 640  7 4
257+271 924  7 4
527+803 1033 7 4
708+052 1057 7 4
 937+31 752  7 4
 73+333 370  7 4
 18+649 1027 7 4
 896+19 789  7 4
029+044 1360 7 4
 21+517 727  7 4
 72+469 991  7 4
836+317 1351 7 4
 26+793 459  7 4
271+468 1036 7 4
074+792 767  7 4
 92+806 637  7 4
511+601 221  7 4
483+135 915  7 4
 66+507 771  7 4
 38+293 475  7 4
816+154 1069 7 4
432+784 721  7 4
 84+833 386  7 4
 69+802 304  7 4
404+998 1303 7

 356+22 675  7 4
345+339 1476 7 4
 13+736 668  7 4
 273+92 401  7 4
187+702 988  7 4
 38+542 328  7 4
 807+24 750  7 4
 83+077 808  7 4
 851+44 202  7 4
178+267 1633 7 4
 58+531 220  7 4
 25+346 695  7 4
  6+231 138  7 4
013+003 610  7 4
 54+718 862  7 4
 89+651 254  7 4
698+394 1389 7 4
 195+15 642  7 4
 86+756 725  7 4
083+876 1058 7 4
014+847 1158 7 4
 097+01 800  7 4
003+954 759  7 4
 31+507 718  7 4
 43+858 892  7 4
 171+66 237  7 4
 61+904 425  7 4
 028+47 894  7 4
 817+33 751  7 4
858+981 1047 7 4
279+603 1278 7 4
 35+497 847  7 4
 45+206 656  7 4
 04+159 991  7 4
653+129 1277 7 4
461+577 939  7 4
  5+719 922  7 4
 654+55 511  7 4
 84+492 342  7 4
717+736 1354 7 4
 64+952 305  7 4
702+039 1137 7 4
887+148 1629 7 4
 14+614 457  7 4
 51+871 193  7 4
 747+25 799  7 4
461+257 916  7 4
731+597 932  7 4
497+558 1649 7 4
 25+376 725  7 4
 466+51 679  7 4
565+109 1466 7 4
357+028 1573 7 4
 52+996 724  7 4
066+379 1633 7 4
162+689 1247 7 4
507+204 1107 7 4
 31+457 767  7 4
239+098 1822 7

729+783 1314 7 4
 453+35 407  7 4
473+008 1174 7 4
825+232 760  7 4
 591+02 215  7 4
 664+34 509  7 4
843+373 721  7 4
 022+48 304  7 4
863+925 897  7 4
 068+48 944  7 4
126+962 890  7 4
514+945 964  7 4
 443+68 430  7 4
  022+8 228  7 4
 335+01 543  7 4
  3+956 662  7 4
 48+706 691  7 4
 493+74 441  7 4
  946+9 658  7 4
 32+718 840  7 4
074+286 1152 7 4
593+092 685  7 4
 421+19 215  7 4
 833+79 435  7 4
 74+928 876  7 4
779+081 1157 7 4
288+346 1525 7 4
 66+007 766  7 4
 78+621 213  7 4
074+981 659  7 4
643+744 793  7 4
613+053 666  7 4
  968+9 878  7 4
 705+11 518  7 4
 265+33 595  7 4
076+189 1651 7 4
 66+019 976  7 4
 11+321 134  7 4
 32+997 822  7 4
208+317 1515 7 4
 63+815 554  7 4
 286+22 704  7 4
 47+306 677  7 4
 236+16 693  7 4
 834+84 486  7 4
658+216 1468 7 4
774+811 595  7 4
153+208 1153 7 4
  495+6 600  7 4
605+314 919  7 4
 56+921 194  7 4
 367+94 812  7 4
 534+64 481  7 4
 85+885 646  7 4
701+984 596  7 4
 152+47 325  7 4
427+034 1154 7 4
 649+72 973  7 4
504+533 740  7

 84+954 507  7 4
669+884 1454 7 4
 403+87 382  7 4
 38+785 670  7 4
 83+954 497  7 4
613+411 430  7 4
 415+27 586  7 4
781+025 707  7 4
 746+43 681  7 4
 095+82 618  7 4
 235+36 595  7 4
782+143 628  7 4
 83+809 946  7 4
763+355 920  7 4
607+352 959  7 4
 31+146 654  7 4
 772+09 367  7 4
 05+474 524  7 4
 08+693 476  7 4
425+554 979  7 4
166+883 1049 7 4
 246+96 711  7 4
 465+37 637  7 4
 23+033 362  7 4
783+849 1335 7 4
 57+285 657  7 4
 78+713 404  7 4
304+984 892  7 4
 59+265 657  7 4
 38+421 207  7 4
727+785 1314 7 4
 826+86 696  7 4
433+004 734  7 4
 75+082 337  7 4
777+022 997  7 4
  239+8 940  7 4
  2+788 889  7 4
 64+225 568  7 4
977+531 914  7 4
 34+594 538  7 4
 411+64 160  7 4
211+527 837  7 4
 53+764 502  7 4
 96+226 691  7 4
 61+474 490  7 4
 839+56 1003 7 4
081+382 463  7 4
725+878 1405 7 4
 889+67 1064 7 4
925+144 970  7 4
 897+67 874  7 4
 37+593 468  7 4
659+745 1503 7 4
977+802 987  7 4
 534+68 521  7 4
  94+84 97   7 4
 515+08 595  7 4
486+671 860  7 4
 81+787 805  7

In [ ]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

In [304]:
x,y

(array([[[ True, False, False, ..., False, False, False],
         [ True, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False,  True, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[ True, False, False, ..., False, False, False],
         [ True, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False,  True, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[ True, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False,  True, ..., False, False, False],
         ...,
         [False,  True, False, ..., False, False, False],
         [False, False, 

In [4]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_____________________________

In [8]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 18s 394us/step - loss: 1.8898 - acc: 0.3228 - val_loss: 1.7981 - val_acc: 0.3428
Q 300+54  T 354  ☒ 109 
Q 297+86  T 383  ☒ 109 
Q 123+188 T 311  ☒ 107 
Q 727+79  T 806  ☒ 107 
Q 279+4   T 283  ☒ 153 
Q 629+85  T 714  ☒ 100 
Q 76+756  T 832  ☒ 107 
Q 831+97  T 928  ☒ 100 
Q 270+619 T 889  ☒ 107 
Q 339+92  T 431  ☒ 109 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 1.7385 - acc: 0.3564 - val_loss: 1.6713 - val_acc: 0.3777
Q 435+446 T 881  ☒ 104 
Q 510+37  T 547  ☒ 225 
Q 870+65  T 935  ☒ 707 
Q 545+700 T 1245 ☒ 102 
Q 894+64  T 958  ☒ 904 
Q 117+20  T 137  ☒ 221 
Q 546+96  T 642  ☒ 705 
Q 159+5   T 164  ☒ 159 
Q 8+140   T 148  ☒ 111 
Q 71+66   T 137  ☒ 177 

------------------

45000/45000 [==============================] - 9s 208us/step - loss: 0.2090 - acc: 0.9523 - val_loss: 0.1904 - val_acc: 0.9531
Q 454+907 T 1361 ☑ 1361
Q 456+1   T 457  ☑ 457 
Q 5+801   T 806  ☑ 806 
Q 218+18  T 236  ☑ 236 
Q 166+922 T 1088 ☒ 1098
Q 7+649   T 656  ☑ 656 
Q 26+293  T 319  ☑ 319 
Q 820+85  T 905  ☑ 905 
Q 671+91  T 762  ☑ 762 
Q 490+938 T 1428 ☒ 1529

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
 1408/45000 [..............................] - ETA: 8s - loss: 0.1693 - acc: 0.9673

KeyboardInterrupt: 